![example]()

# Predicting-Most-Valuable-Home-Projects-In-King-County-Analysis

# Introduction

The King County Housing Data Set contains information about the size, location, condition, and other features of houses in King County. A full description of the dataset's columns can be found below. The aim of this project is to develop a linear regression model than can predict a house's price as accurately as possible.

# Business Problem

A client in King County, WA wants to advise homeowners on home improvement projects that will add to the sale value of their homes.

# Analysis Questions

This analysis will seek to answer three questions about the data:

Question 1: Will enclosing a porch increase the sale price of a home?

Question 2: Is converting a garage to a bedroom a good way to increase the sale price of a home?

Questin 3: Will upgrading to a forced-air heating system increase the sale price of a home?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn import datasets, linear_model
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('../../data/raw/EXTR_RPSale.csv')
df.head()

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,2857854,198920,1430,03/28/2017,0,20170410000541,,,,,...,3,7,15,N,N,N,N,16,2,20 31
1,2743355,638580,110,07/14/2015,190000,20150715002686,,,,,...,3,6,3,N,N,N,N,1,8,15
2,2999169,919715,200,07/08/2019,192000,20190712001080,,,,,...,3,2,3,N,N,N,N,1,3,
3,2841697,894677,240,12/21/2016,818161,20161228000896,,,,,...,2,6,3,N,N,N,N,1,8,
4,2826129,445872,260,10/03/2016,0,20161004000511,,,,,...,3,2,15,N,N,N,N,18,3,18 31


In [104]:
zerocontent = df['AFCurrentUseLand'].str.contains('N')
zerocontent

0          True
1          True
2          True
3          True
4          True
          ...  
351062     True
351063     True
351064     True
351065     True
351066    False
Name: AFCurrentUseLand, Length: 351067, dtype: bool

In [105]:
forestland = zerocontent.loc[zerocontent != False]
len(forestland)

312511

In [106]:
len(zerocontent) - len(forestland)

38556

In [90]:
details = df.apply(lambda x: True if "N" in df['AFForestLand'] else False , axis=1)
details

0         False
1         False
2         False
3         False
4         False
          ...  
351062    False
351063    False
351064    False
351065    False
351066    False
Length: 351067, dtype: bool

In [107]:
to_drop = ['AFForestLand','AFCurrentUseLand','AFNonProfitUse', 'AFHistoricProperty']
df = df.drop(to_drop, axis=1)

In [108]:
df.head()

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot,PlatBlock,SellerName,BuyerName,PropertyType,PrincipalUse,SaleInstrument,SaleReason,PropertyClass,SaleWarning
0,2857854,198920,1430,03/28/2017,0,20170410000541,,,,,,,AD APARTMENTS LLC ...,SEATTLE CITY OF ...,3,7,15,16,2,20 31
1,2743355,638580,110,07/14/2015,190000,20150715002686,,,,,,,GINGRICH AUDREY B ...,NATION FRED I ...,3,6,3,1,8,15
2,2999169,919715,200,07/08/2019,192000,20190712001080,,,,,,,WAGNERESTATES LLC ...,SCHAFFER CORBIN ...,3,2,3,1,3,
3,2841697,894677,240,12/21/2016,818161,20161228000896,,,,,,,QUADRANT CORPORATION ...,KANG JING ...,2,6,3,1,8,
4,2826129,445872,260,10/03/2016,0,20161004000511,,,,,,,MARCH CARL J + SUSAN A ...,MARCH CARL J + SUSAN A REVOCABLE LIVING TRUST ...,3,2,15,18,3,18 31


In [65]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181510 entries, 0 to 181509
Data columns (total 50 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Major               181510 non-null  int64  
 1   Minor               181510 non-null  int64  
 2   BldgNbr             181510 non-null  int64  
 3   NbrLivingUnits      181510 non-null  int64  
 4   Address             181510 non-null  object 
 5   BuildingNumber      181510 non-null  object 
 6   Fraction            181510 non-null  object 
 7   DirectionPrefix     181146 non-null  object 
 8   StreetName          181510 non-null  object 
 9   StreetType          181510 non-null  object 
 10  DirectionSuffix     181146 non-null  object 
 11  ZipCode             154594 non-null  object 
 12  Stories             181510 non-null  float64
 13  BldgGrade           181510 non-null  int64  
 14  BldgGradeVar        181510 non-null  int64  
 15  SqFt1stFloor        181510 non-nul

In [5]:
df2 = pd.read_csv('../../data/raw/EXTR_ResBldg.csv')
df2.head()

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,9800,720,1,1,27719 SE 26TH WAY 98075,27719,,SE,26TH,WAY,...,0,0,0,2001,0,0,0,0,3,0
1,9802,140,1,1,2829 277TH TER SE 98075,2829,,,277TH,TER,...,0,0,0,2004,0,0,0,0,3,0
2,9830,20,1,1,1715 298TH CRESENT SE,1715,,,298TH CRESENT,,...,0,0,0,2017,0,0,0,0,3,0
3,9830,160,1,1,1861 297TH WAY SE 98024,1861,,,297TH,WAY,...,0,0,0,2013,0,0,0,0,3,0
4,10050,180,1,1,35410 25TH PL S 98003,35410,,,25TH,PL,...,0,0,0,1994,0,0,0,0,4,0


In [67]:
parcel = pd.read_csv('../../data/raw/EXTR_Parcel.csv')
parcel.tail()

,Unnamed: 0,Major,Minor,PropName,PlatName,PlatLot,PlatBlock,Range,Township,Section,...,SeismicHazard,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems
205194,614815,168940,1527,,COLLINS ADD,4,90,4,25,19,...,N,N,N,N,N,N,N,N,N,N
205195,614818,602150,3701,,NELSON H E ADD,25,18,3,25,2,...,N,N,N,N,N,N,N,N,N,N
205196,614820,42100,35,,BALCHS ALBERT PRESIDENT PARK NO. 04,7,1,5,23,9,...,N,N,N,N,N,N,N,N,N,N
205197,614821,613160,45,,NORTH AUBURN ADD,9,1,5,21,7,...,N,N,N,N,N,N,N,N,N,N
205198,614822,881990,840,,UNIVERSITY HOME TRS,4 & 13,8,4,25,10,...,N,N,N,N,N,N,N,N,N,N


In [7]:
df4 = pd.read_csv('../../data/raw/EXTR_LookUp.csv')
df4.head()

,LUType,LUItem,LUDescription
0,1,1,LAND ONLY ...
1,1,10,Land with new building ...
2,1,11,"Household, single family units ..."
3,1,12,"Multiple family residence (Residential, 2-4 un..."
4,1,13,"Multiple family residence (Residential, 5+ uni..."


In [44]:
(df4.iloc[[3]])

,LUType,LUItem,LUDescription
3,1,12,"Multiple family residence (Residential, 2-4 un..."


In [43]:
display(df4.loc[2])

LUType                                                           1
LUItem                                                          11
LUDescription    Household, single family units                ...
Name: 2, dtype: object

In [58]:
with pd.option_context('display.max_colwidth', None):
  display(df4.head(10))

,LUType,LUItem,LUDescription
0,1,1,LAND ONLY
1,1,10,Land with new building
2,1,11,"Household, single family units"
3,1,12,"Multiple family residence (Residential, 2-4 units)"
4,1,13,"Multiple family residence (Residential, 5+ units)"
5,1,14,Residential condominiums
6,1,15,Mobile home parks or courts
7,1,16,Hotels/motels
8,1,17,Institutional lodging
9,1,18,All other residential not elsewhere coded


In [71]:
multiple = df4['LUDescription'].str.contains('Multiple')
multiple

0       False
1       False
2       False
3        True
4        True
        ...  
1203    False
1204    False
1205    False
1206    False
1207    False
Name: LUDescription, Length: 1208, dtype: bool

In [73]:
with pd.option_context('display.max_colwidth', None):
    display(df4[multiple])

,LUType,LUItem,LUDescription
3,1,12,"Multiple family residence (Residential, 2-4 units)"
4,1,13,"Multiple family residence (Residential, 5+ units)"
477,118,587,"Shell, Multiple Residence (587)"
508,118,784,"Shell, Multiple Res. (Sen. Citizen) (784)"


In [70]:
with pd.option_context('display.max_colwidth', None):
  display(df4['LUDescription'].str.contains 'Multiple')

0       False
1       False
2       False
3       False
4       False
        ...  
1203    False
1204    False
1205    False
1206    False
1207    False
Name: LUDescription, Length: 1208, dtype: bool